In [1]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer



In [2]:
# fetch data

x_train = pd.read_csv("AMF_train_X.csv")
y_train = pd.read_csv("AMF_train_Y.csv")
x_test = pd.read_csv("AMF_test_X.csv")


In [3]:
# dictionnary {Trader:TYPE}

type_d = {}
for index, row in y_train.iterrows():
    type_d[row['Trader']] = row['type']

In [17]:
# {Custom train data}

x_train2 = x_train.iloc[:60467,:]
x_train2 = x_train2.dropna()
y_train2 = [type_d[x] for x in x_train2['Trader']]
x_train2 = x_train2.drop(columns=['Index','Share','Day','Trader'])

x_test2 = x_train.iloc[60467:,:]
x_test2 = x_test2.dropna()

y_test2 = [type_d[x] for x in x_test2['Trader']]
x_test2 = x_test2.drop(columns=['Index','Share','Day','Trader'])


In [5]:

test = x_train2.iloc[:500,:]
test_y = y_train2[:500]

svclassifier = SVC(kernel='linear')
svclassifier.fit(test, test_y)

SVC(kernel='linear')

In [19]:
# x_test2 = x_test.drop(columns=['Index','Share','Day','Trader']).dropna()

y_pred = svclassifier.predict(x_test2)
print(classification_report(y_test2,y_pred))


              precision    recall  f1-score   support

         HFT       0.68      0.23      0.34      3460
         MIX       0.81      0.79      0.80     14848
     NON HFT       0.38      0.88      0.53      1857

    accuracy                           0.71     20165
   macro avg       0.62      0.64      0.56     20165
weighted avg       0.74      0.71      0.70     20165



In [24]:

x_test2 = x_train.iloc[60467:,:]
x_test2 = x_test2.dropna()


names = x_test2['Trader'].unique()

convention = {'HFT': 0, 'NON HFT': 1, 'MIX': 2} 
result = {e: [0,0,0] for e in names}
i = 0
for index, row in x_test2.iterrows():
    result[row['Trader']][convention[y_pred[i]]] += 1
    i+=1

data=[]
for trader in names:
    n = sum(result[trader])
    if result[trader][0]>0.85*n:
        data.append([trader, 'HFT'])
    elif result[trader][2]>0.5*n:
        data.append([trader, 'MIX'])
    else:
        data.append([trader, 'NON HFT'])

export_df = pd.DataFrame(data, columns = ['Trader', 'type'])
print(export_df)
export_df.to_csv(r'test.csv',index=False)


        Trader     type
0   Trader_310      MIX
1   Trader_312  NON HFT
2   Trader_329      MIX
3   Trader_330      MIX
4   Trader_338      MIX
5   Trader_341      MIX
6    Trader_35  NON HFT
7   Trader_358      MIX
8    Trader_38      MIX
9   Trader_380  NON HFT
10  Trader_395      MIX
11   Trader_40      MIX
12  Trader_424  NON HFT
13  Trader_430  NON HFT
14  Trader_431      MIX
15  Trader_438      MIX
16  Trader_442  NON HFT
17  Trader_451      MIX
18  Trader_453      MIX
19  Trader_461      MIX
20   Trader_54      MIX


In [27]:
test_true = pd.read_csv("AMF_train_Y.csv")
print(len(test_true))
test_true=test_true.drop(test_true[test_true['Trader'] not in names].index)
print(len(test_true))


86
<ipython-input-27-9762d601d123>:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  test_true=test_true.drop(test_true[test_true['Trader'] not in names].index)


ValueError: ('Lengths must match to compare', (86,), (21,))